In [0]:
import pandas as pd

df_sheet = spark.read.option("header",True).csv("dbfs:/FileStore/sheet_id.csv")
#df_sheet = spark.read.option("header",True).csv("/FileStore/sheet_id.csv")
df_sheet_search = df_sheet.toPandas() # pandas dataframe
#sheet_str = str(tuple(df_sheet_search.loc[[0,1],['SHEET_ID']]['SHEET_ID'].to_list()))
sheet_str = str(tuple(df_sheet_search['SHEET_ID'].to_list()))

df_sheet_search['L6K_SHEET_ID'] = df_sheet_search['SHEET_ID']
df_sheet_search.set_index('SHEET_ID', inplace=True)
df_sheet_search

Out[1]:

,DVT,L6K_SHEET_ID
SHEET_ID,,
EE01NB2K50P07,4-5_mon,EE01NB2K50P07
EE01NB2K50P08,4-5_mon,EE01NB2K50P08
EE01NB2K50P09,4-5_mon,EE01NB2K50P09
EE01NB2K50P10,4-5_mon,EE01NB2K50P10
EE01NB2K50P11,4-5_mon,EE01NB2K50P11
...,...,...
EE02N20590P26,4-7-2_MON2_T10V2+MEV1,EE02N20590P26
EE02N20600P01,4-7-2_MON2_T10V2+MEV1,EE02N20600P01
EE02N20600P02,4-7-2_MON2_T10V2+MEV1,EE02N20600P02


In [0]:
# SQL SEARCH AND CREATE DATAFRAME 查詢與產生資料表:

sql_string = f'''
  select SOURCE_SHEET_ID as L6K_SHEET_ID, TARGET_SHEET_ID as L6B_SHEET_ID  
  from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
  where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
  group by SOURCE_SHEET_ID,TARGET_SHEET_ID  
'''

#print(sql_string)

df_SHEET_DF = spark.sql(sql_string).toPandas()
df_SHEET_INDEX = df_SHEET_DF.copy()
df_SHEET_INDEX['SHEET_INDEX'] = df_SHEET_INDEX['L6B_SHEET_ID']
df_SHEET_INDEX.set_index('SHEET_INDEX', inplace=True)
if len(df_SHEET_INDEX)>0 : 
    df_sheet_search.loc[df_sheet_search.index.isin(df_SHEET_INDEX.index), ['L6B_SHEET_ID']] = df_SHEET_INDEX['L6B_SHEET_ID']
    df_sheet_search.loc[df_sheet_search.index.isin(df_SHEET_INDEX.index), ['L6K_SHEET_ID']] = df_SHEET_INDEX['L6K_SHEET_ID']
    df_SHEET_INDEX = df_SHEET_DF.copy()
    df_SHEET_INDEX['SHEET_INDEX'] = df_SHEET_INDEX['L6K_SHEET_ID']
    df_SHEET_INDEX.set_index('SHEET_INDEX', inplace=True)
    df_sheet_search.loc[df_sheet_search.index.isin(df_SHEET_INDEX.index), ['L6B_SHEET_ID']] = df_SHEET_INDEX['L6B_SHEET_ID']
    df_sheet_search.loc[df_sheet_search.index.isin(df_SHEET_INDEX.index), ['L6K_SHEET_ID']] = df_SHEET_INDEX['L6K_SHEET_ID']

df_sheet_search

Out[2]:

,DVT,L6K_SHEET_ID,L6B_SHEET_ID
SHEET_ID,,,
EE01NB2K50P07,4-5_mon,EE1NB2K5G,EE01NB2K50P07
EE01NB2K50P08,4-5_mon,EE1NB2K5H,EE01NB2K50P08
EE01NB2K50P09,4-5_mon,EE1NB2K5J,EE01NB2K50P09
EE01NB2K50P10,4-5_mon,EE1NB2K5K,EE01NB2K50P10
EE01NB2K50P11,4-5_mon,EE1NB2K5L,EE01NB2K50P11
...,...,...,...
EE02N20590P26,4-7-2_MON2_T10V2+MEV1,EE2N20592,EE02N20590P26
EE02N20600P01,4-7-2_MON2_T10V2+MEV1,EE2N2060A,EE02N20600P01
EE02N20600P02,4-7-2_MON2_T10V2+MEV1,EE2N2060B,EE02N20600P02


In [0]:
# SQL SEARCH AND CREATE DATAFRAME 查詢與產生資料表:

sql_string = f'''
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, 
  case when A.PEP_LEVEL is null then ' ' else A.PEP_LEVEL end as PEP_LEVEL, A.OP_ID, case when A.OP_SEQ is null then ' ' when length(A.OP_SEQ)=2 then '000'||A.OP_SEQ when length(A.OP_SEQ)=3 then '00'||A.OP_SEQ when length(A.OP_SEQ)=4 then '0'||A.OP_SEQ else A.OP_SEQ end as OP_SEQ, 
  case when A.EQP_ID is null then ' ' else A.EQP_ID end as EQP_ID, case when A.CHAMBER_ID_LIST is null then ' ' when length(A.CHAMBER_ID_LIST)<2 then ' ' else A.CHAMBER_ID_LIST end as CHAMBER_ID_LIST, 
  A.RECIPE_ID, A.ACTUAL_PPID, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6b_array_h_sheet_oper_ods A 
  where (A.SHEET_ID_CHIP_ID in 
  ( 
    select SOURCE_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by SOURCE_SHEET_ID 
  )  
  or A.SHEET_ID_CHIP_ID in 
  ( 
    select TARGET_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by TARGET_SHEET_ID 
  )) 
  and (substr(A.OP_ID,6,1) in ('-','+') or substr(A.OP_ID,5,1) in ('-','+') or substr(A.OP_ID,4,1) in ('-','+') or substr(A.OP_ID,3,1) in ('-','+'))
  
  union 
  
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, 
  case when A.PEP_LEVEL is null then ' ' else A.PEP_LEVEL end as PEP_LEVEL, A.OP_ID, case when A.OP_SEQ is null then ' ' when length(A.OP_SEQ)=2 then '000'||A.OP_SEQ when length(A.OP_SEQ)=3 then '00'||A.OP_SEQ when length(A.OP_SEQ)=4 then '0'||A.OP_SEQ else A.OP_SEQ end as OP_SEQ, 
  case when A.EQP_ID is null then ' ' else A.EQP_ID end as EQP_ID, case when A.CHAMBER_ID_LIST is null then ' ' when length(A.CHAMBER_ID_LIST)<2 then ' ' else A.CHAMBER_ID_LIST end as CHAMBER_ID_LIST, 
  A.RECIPE_ID, A.ACTUAL_PPID, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6k_array_h_sheet_oper_ods A 
  where (A.SHEET_ID_CHIP_ID in 
  ( 
    select SOURCE_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by SOURCE_SHEET_ID 
  )  
  or A.SHEET_ID_CHIP_ID in 
  ( 
    select TARGET_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by TARGET_SHEET_ID 
  )) 
  and (substr(A.OP_ID,5,1) in ('-','+') or substr(A.OP_ID,4,1) in ('-','+') or substr(A.OP_ID,3,1) in ('-','+'))
'''

#print(sql_string)

df_sheet_operator = spark.sql(sql_string).toPandas()

#df_sheet_operator.write.csv("dbfs:/FileStore/result.csv")

df_sheet_operator

Out[3]:

,LOT_ID,SHEET_ID,MFG_DAY,MODEL_NO,PRODUCT_CODE,MAIN_ROUTE_ID,ROUTE_ID,PEP_LEVEL,OP_ID,OP_SEQ,EQP_ID,CHAMBER_ID_LIST,RECIPE_ID,ACTUAL_PPID,CASSETTE_ID,REWORK_FLAG,ALARM_FLAG,LOGON_TIME,LOGOFF_TIME,HOLD_TIME,QUEUE_TIME,OP_PROCESS_TIME,OP_CYCLE_TIME,EQP_RUN_ID_START_TIME,EQP_RUN_ID_END_TIME
0,EE12G303Y0,EE02N12790P16,2022-03-04,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP1,UKE-STO,03000,WR-10C,C01,0000060100000000,0000060100000000,SF5A1521,N,None,2022-03-04 19:20:07,2022-03-04 20:18:40,0,125998,3513,129511,2022-03-04 19:49:20,NaT
1,EE12G305Y0,EE01NB2K50P25,2022-03-05,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP2,PL2-IEX,03091,CAJ-03C,C01,1620016200000000,1620016200000000,SF5C1717,N,None,2022-03-05 13:53:08,2022-03-05 21:34:58,0,69916,27710,97626,2022-03-05 18:20:31,NaT
2,EE12G301Y0,EE02N12070P24,2022-03-06,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP3,BP2-CVD,03133,VDN-07C,CHD,P2EBBP20PV200000,P2EBBP20PV200000,SF550660,N,None,2022-03-06 09:26:41,2022-03-06 10:46:39,0,2608,4798,7406,2022-03-06 09:48:35,NaT
3,EE12G306Y0,EE01NB2K50P11,2022-03-06,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP2,PL2-ANI,03112,XO-07C,C01,0000000600000000,0000000600000000,SA075405,N,None,2022-03-06 11:24:02,2022-03-06 12:58:54,0,39352,5692,45044,2022-03-06 11:34:44,NaT
4,EE12G302Y0,EE02N12820P11,2022-03-06,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP3,BP2-CVD,03133,VDN-07C,CHA,P2EBBP20PV200000,P2EBBP20PV200000,SF5C1605,N,None,2022-03-06 13:59:36,2022-03-06 15:43:21,0,8273,6225,14498,2022-03-06 14:47:30,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69854,EE2N205500,EE2N2055N,2022-03-18,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,-,ARY-FI,08888,ANDMY200,,0999,None,AA0815,N,None,2022-03-18 13:53:23,2022-03-18 13:53:23,0,8683,0,8683,NaT,NaT
69855,EE2N205900,EE2N2059E,2022-03-18,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP9,BP2-CVD,00960,ANCVD100,P7,STD-BP2LH-2000,STD-BP2LH-2000,FA0373,N,None,2022-03-18 16:44:08,2022-03-18 18:13:22,0,1753,5354,7107,2022-03-18 17:09:47,NaT
69856,EE2N13S100,EE2N13S1C,2022-03-19,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,-,ARY-FI,08888,ANDMY200,,0999,None,AA0983,N,None,2022-03-19 09:34:00,2022-03-19 09:34:00,0,455,0,455,NaT,NaT
69857,EE2N205900,EE2N2059U,2022-03-19,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,ANDMY200,ECS-BANK,09009,ANDMY200,,999,None,FA0373,N,None,2022-03-19 11:21:07,2022-03-19 11:21:25,0,8041,18,8059,NaT,NaT


In [0]:
df_sheet_operator[df_sheet_operator['OP_SEQ']=='00210']

Out[8]:

,LOT_ID,SHEET_ID,MFG_DAY,MODEL_NO,PRODUCT_CODE,MAIN_ROUTE_ID,ROUTE_ID,PEP_LEVEL,OP_ID,OP_SEQ,EQP_ID,CHAMBER_ID_LIST,RECIPE_ID,ACTUAL_PPID,CASSETTE_ID,REWORK_FLAG,ALARM_FLAG,LOGON_TIME,LOGOFF_TIME,HOLD_TIME,QUEUE_TIME,OP_PROCESS_TIME,OP_CYCLE_TIME,EQP_RUN_ID_START_TIME,EQP_RUN_ID_END_TIME
4733,EE2N128400,EE2N1284U,2022-01-18,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0930,N,None,2022-01-18 20:29:05,2022-01-18 22:17:50,0,79034,6525,85559,2022-01-18 20:36:34,NaT
4751,EE2N13S400,EE2N13S4J,2022-02-09,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0175,N,None,2022-02-09 18:31:24,2022-02-09 21:24:18,0,22349,10374,32723,2022-02-09 19:45:43,NaT
5142,EE2N158300,EE2N1583L,2022-02-06,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0387,N,None,2022-02-06 06:49:34,2022-02-06 11:51:05,0,3183,18091,21274,2022-02-06 10:12:49,NaT
5145,EE2N143000,EE2N1430C,2022-02-07,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG200,,L205,L205,AA0240,N,None,2022-02-07 22:36:29,2022-02-08 03:10:00,0,53038,16411,69449,2022-02-08 00:59:46,NaT
5146,EE2N205500,EE2N2055J,2022-02-09,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0815,N,None,2022-02-09 05:53:37,2022-02-09 12:58:00,0,58968,25463,84431,2022-02-09 11:15:50,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67080,EE2N128400,EE2N1284V,2022-01-18,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0930,N,None,2022-01-18 20:29:05,2022-01-18 22:17:50,0,79034,6525,85559,2022-01-18 20:36:34,NaT
67102,EE2N205600,EE2N20562,2022-02-06,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG200,,L205,L205,AA1079,N,None,2022-02-07 04:14:14,2022-02-07 05:53:13,0,73210,5939,79149,2022-02-07 04:14:32,NaT
67283,EE2N205900,EE2N2059L,2022-02-07,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG200,,L205,L205,FA0373,N,None,2022-02-08 02:21:49,2022-02-08 05:58:56,0,62143,13027,75170,2022-02-08 04:18:49,NaT
67291,EE2N205800,EE2N2058N,2022-02-10,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG200,,L205,L205,AA0689,N,None,2022-02-10 22:41:22,2022-02-11 02:49:26,0,114461,14884,129345,2022-02-11 01:09:43,NaT


In [0]:
%sql
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, 
  case when A.PEP_LEVEL is null then ' ' else A.PEP_LEVEL end as PEP_LEVEL, A.OP_ID, case when A.OP_SEQ is null then ' ' when length(A.OP_SEQ)=2 then '000'||A.OP_SEQ when length(A.OP_SEQ)=3 then '00'||A.OP_SEQ when length(A.OP_SEQ)=4 then '0'||A.OP_SEQ else A.OP_SEQ end as OP_SEQ, 
  case when A.EQP_ID is null then ' ' else A.EQP_ID end as EQP_ID, case when A.CHAMBER_ID_LIST is null then ' ' when length(A.CHAMBER_ID_LIST)<2 then ' ' else A.CHAMBER_ID_LIST end as CHAMBER_ID_LIST, 
  A.RECIPE_ID, A.ACTUAL_PPID, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6k_array_h_sheet_oper_ods A 
  where A.SHEET_ID_CHIP_ID in ('EE2N2055N', 'EE02N12820P11')  
  
  union 
  
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, 
  case when A.PEP_LEVEL is null then ' ' else A.PEP_LEVEL end as PEP_LEVEL, A.OP_ID, case when A.OP_SEQ is null then ' ' when length(A.OP_SEQ)=2 then '000'||A.OP_SEQ when length(A.OP_SEQ)=3 then '00'||A.OP_SEQ when length(A.OP_SEQ)=4 then '0'||A.OP_SEQ else A.OP_SEQ end as OP_SEQ, 
  case when A.EQP_ID is null then ' ' else A.EQP_ID end as EQP_ID, case when A.CHAMBER_ID_LIST is null then ' ' when length(A.CHAMBER_ID_LIST)<2 then ' ' else A.CHAMBER_ID_LIST end as CHAMBER_ID_LIST, 
  A.RECIPE_ID, A.ACTUAL_PPID, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6b_array_h_sheet_oper_ods A 
  where A.SHEET_ID_CHIP_ID in ('EE2N2055N', 'EE02N12820P11') 
  order by A.LOGOFF_TIME  

LOT_ID,SHEET_ID,MFG_DAY,MODEL_NO,PRODUCT_CODE,MAIN_ROUTE_ID,ROUTE_ID,PEP_LEVEL,OP_ID,OP_SEQ,EQP_ID,CHAMBER_ID_LIST,RECIPE_ID,ACTUAL_PPID,CASSETTE_ID,REWORK_FLAG,ALARM_FLAG,LOGON_TIME,LOGOFF_TIME,HOLD_TIME,QUEUE_TIME,OP_PROCESS_TIME,OP_CYCLE_TIME,EQP_RUN_ID_START_TIME,EQP_RUN_ID_END_TIME
EE2N205500,EE2N2055N,2022-02-01T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-CLA,00100,ANCLI100,,CN5TV1,CN5TV1,AA0815,N,null,2022-02-01T16:04:52.000+0000,2022-02-01T16:35:47.000+0000,0,0,1855,0,2022-02-01T16:11:08.000+0000,null
EE2N205500,EE2N2055N,2022-02-01T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-SPT,00110,ANSPG100,CA52,M0500,M0500,AA0815,N,null,2022-02-01T19:28:12.000+0000,2022-02-01T20:32:57.000+0000,0,10345,3885,14230,2022-02-01T19:50:48.000+0000,null
EE2N205500,EE2N2055N,2022-02-01T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-IEX,00120,ANIEX300,,701 7001 7001,7001,AA0815,N,null,2022-02-02T00:06:55.000+0000,2022-02-02T01:39:45.000+0000,0,12838,5570,18408,2022-02-02T00:58:15.000+0000,null
EE2N205500,EE2N2055N,2022-02-03T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-ICP,00130,ANWMA200,,NB-SM-WMA-T5,NB-SM-WMA-T5,AA0815,N,null,2022-02-04T05:45:37.000+0000,2022-02-04T07:20:10.000+0000,0,135353,5673,141026,2022-02-04T06:41:37.000+0000,null
EE2N205500,EE2N2055N,2022-02-04T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-STO,00140,ANSTO100,,0008,0008,AA0815,N,null,2022-02-05T04:00:29.000+0000,2022-02-05T04:47:17.000+0000,0,74419,2808,77227,2022-02-05T04:21:08.000+0000,null
EE2N205500,EE2N2055N,2022-02-09T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ELA,00210,ANELG300,,L205,L205,AA0815,N,null,2022-02-09T05:53:37.000+0000,2022-02-09T12:58:00.000+0000,0,58968,25463,84431,2022-02-09T11:15:50.000+0000,null
EE2N205500,EE2N2055N,2022-02-10T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-IEX,00220,ANIEX400,,702 7002 7002,7002,AA0815,N,null,2022-02-10T12:56:59.000+0000,2022-02-10T14:14:37.000+0000,0,86339,4658,90997,2022-02-10T13:31:21.000+0000,null
EE2N205500,EE2N2055N,2022-02-10T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-ICP,00230,ANICP100,,T127SUN01-POLY,T127SUN01-POLY,AA0815,N,null,2022-02-10T17:36:36.000+0000,2022-02-10T18:17:26.000+0000,0,5441,2450,7891,2022-02-10T17:39:38.000+0000,null
EE2N205500,EE2N2055N,2022-02-10T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,POLY-STO,00240,ANSTO200,,0008,0008,AA0815,N,null,2022-02-10T19:11:17.000+0000,2022-02-10T20:10:13.000+0000,0,3231,3536,6767,2022-02-10T19:42:20.000+0000,null
EE2N205500,EE2N2055N,2022-02-10T00:00:00.000+0000,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP2,NCD-IID,00250,ANIID300,,B18K6_00E11,B18K6_00E11,AA0815,N,null,2022-02-10T21:25:53.000+0000,2022-02-10T22:02:23.000+0000,0,4540,2190,6730,2022-02-10T21:27:20.000+0000,null


In [0]:
import pandas as pd
ARRAY_OPERATOR_DF = pd.merge(df_sheet_operator, df_SHEET_DF, left_on="SHEET_ID", right_on="L6K_SHEET_ID", how="inner").append(pd.merge(df_sheet_operator, df_SHEET_DF, left_on="SHEET_ID", right_on="L6B_SHEET_ID", how="inner"))[['L6K_SHEET_ID', 'L6B_SHEET_ID', 'SHEET_ID','PEP_LEVEL','OP_SEQ','OP_ID', 'EQP_ID','CHAMBER_ID_LIST','RECIPE_ID', 'LOGOFF_TIME','HOLD_TIME','QUEUE_TIME','OP_PROCESS_TIME','OP_CYCLE_TIME']].sort_values(by=["L6K_SHEET_ID", "LOGOFF_TIME"],ascending=[True,False]).reset_index(drop=True)

ARRAY_OPERATOR_DF['opid_key'] = ARRAY_OPERATOR_DF['OP_SEQ'].astype(str) + '_' + ARRAY_OPERATOR_DF['PEP_LEVEL'].astype(str) + '_' + ARRAY_OPERATOR_DF['OP_ID'].astype(str)
ARRAY_OPERATOR_DF['eqid_key'] = ARRAY_OPERATOR_DF['EQP_ID'].astype(str) + '_' + ARRAY_OPERATOR_DF['CHAMBER_ID_LIST'].astype(str)

ARRAY_OPERATOR_DF.pivot_table(values='eqid_key', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')

#ARRAY_OPERATOR_DF.pivot_table(values='RECIPE_ID', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')
#ARRAY_OPERATOR_DF.pivot_table(values='LOGOFF_TIME', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')
#ARRAY_OPERATOR_DF.pivot_table(values='QUEUE_TIME', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')
#ARRAY_OPERATOR_DF.pivot_table(values='OP_PROCESS_TIME', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')
#ARRAY_OPERATOR_DF.pivot_table(values='OP_CYCLE_TIME', index=['L6K_SHEET_ID','L6B_SHEET_ID'], columns='opid_key', aggfunc='sum')

Out[5]:

,opid_key,00100_PEP1_SM-CLA,00110_PEP1_SM-SPT,00120_PEP1_SM-IEX,00130_PEP1_SM-ICP,00140_PEP1_SM-STO,00250_PEP2_NCD-IID,00260_PEP2_GI-CVD,00300_PEP3_GE-SPT,00310_PEP3_GE-IEX,00320_PEP3_GE-ICP,00330_PEP3_GE-BID,00340_PEP3_GE-ICP2,00350_PEP3_GE-STO,00360_PEP3_GE-BID2,00370_PEP3_GE-AIO2,00390_PEP4_GI2-CVD,00400_PEP4_M15-SPT,00410_PEP4_M15-IEX,00420_PEP4_M15-ICP,00450_PEP4_M15-STO,00500_PEP5_ILD-CVD,00510_PEP5_ILD-CLA,00540_PEP5_ILD-RTP,00550_PEP5_ILD-IEX,00560_PEP5_ILD-ICP,00570_PEP5_ILD-STO,00600_PEP6_SD-SPT,00610_PEP6_SD-IEX,00620_PEP6_SD-ICP,00630_PEP6_SD-STO,00640_PEP6_SD-ASO,00650_PEP6_SD-ANI,00700_PEP7_PL-IEX,00705_PEP7_PL-USC,00710_PEP7_PL-ANI,00800_PEP8_BP-CVD,00805_PEP8_BP-STO0,00810_PEP8_BP-IEX,00820_PEP8_BP-ICP,00830_PEP8_BP-STO,...,03863_PEP12_M6-SPT,03870_PEP12_M6-IEX,03891_PEP12_M6-WMA,03898_PEP12_M6-STO,03905_PEP12_M6-ASO,04024_PEP14_PV4-CVD,04031_PEP14_PL5-IEX,04052_PEP14_PL5-ANI,04080_PEP15_BP6-CVD,04087_PEP15_BP6-IEX,04108_PEP15_BP6-RIE,04115_PEP15_BP6-STO,04129_PEP15_BP6-ASO,04157_PEP15_BP6-EB,04711_PEP4_GI2+CVD,05051_PEP5_ILD+CVD,05231_PEP5_ILD+QSTO,05531_PEP5_ILD+QCL2,06111_PEP6_SD+STO,06120_PEP3_BP2+STO,06125_PEP3_BP2+IEX,06130_PEP4_M4+STO,06131_PEP6_SD+IEX,06135_PEP4_M4+IEX,06160_PEP7_M5+STO,06165_PEP7_M5+IEX,07071_PEP7_PL+CLA,08011_PEP8_BP+CVD,08051_PEP8_BP+PLR,08231_PEP8_BP+IEX,08888_-_ARY-FI,08900_PEP10_ARY-FI,08900_PEP15_ARY-FI,09009_ANDMY200_ECS-BANK,09211_PEP9_M33+STO,09231_PEP9_M33+IEX,09955_PEP9_BP2+QSTO,09968_PEP9_BP2+ANI,10000_-_ARY-APK,11000_ _ARY-SHIP
L6K_SHEET_ID,L6B_SHEET_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
EE1NB2K51,EE01NB2K50P25,ANCLI100_,ANSPG100_CA42,ANIEX300_,ANWMA200_,ANSTO100_,ANIID300_,ANCVD600_P3,ANSPG200_CA52,ANIEX300_,ANICP600_,ANIID300_,ANICP500_,ANSTO100_,ANIID300_,ANMOR800_,ANCVD400_P7,ANSPG200_CA42,ANIEX200_,ANICP600_,ANSTO400_,ANCVD400_P5,ANCLA200_,ANRTP100_,ANIEX800_,ANICP800_,ANSTO400_,ANSPS200_CA42,ANIEX500_,ANICP300_,ANSTO300_,ANMOR800_,ANANS200_,ANIEX100_,ANUSC100_,ANANO100_,ANCVD100_P7,ANCLA100_,ANIEX900_,ANICPA00_,ANSTO200_,...,NaN,NaN,NaN,WR-07C_C01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANDMY200_,FI-DUMY4_,FI-DUMY4_,ANDMY200_,NaN,NaN,NaN,NaN,ANAPK100_,_
EE1NB2K52,EE01NB2K50P26,ANCLI100_,ANSPG100_CA42,ANIEX300_,ANWMA200_,ANSTO100_,ANIID300_,ANCVD600_P4,ANSPG200_CA42,ANIEX300_,ANICP600_,ANIID300_,ANICP500_,ANSTO100_,ANIID300_,ANMOR800_,ANCVD400_P4,ANSPG200_CA52,ANIEX200_,ANICP600_,ANSTO400_,ANCVD400_P6,ANCLA200_,ANRTP100_,ANIEX800_,ANICP800_,ANSTO400_,ANSPS200_CA52,ANIEX500_,ANICP300_,ANSTO300_,ANMOR800_,ANANS200_,ANIEX100_,ANUSC100_,ANANO100_,ANCVD100_P2,ANCLA100_,ANIEX900_,ANICPA00_,ANSTO200_,...,SPS-04C_AC01,CAR-24C_C01,WA-05C_C01,WR-07C_C01,NaN,VDN-07C_CHE,CAJ-03C_C01,XO-07C_C02,VDN-07C_CHE,CAR-11C_C01,DRG-03C_CC01,WR-07C_C01,NaN,EB-16C_C01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANDMY200_,FI-DUMY4_,FI-DUMY4_,ANDMY200_,NaN,NaN,NaN,NaN,ANAPK100_,_
EE1NB2K5G,EE01NB2K50P07,ANCLI100_,ANSPG100_CA42,ANIEX300_,ANWMA200_,ANSTO100_,ANIID300_,ANCVD600_P7,ANSPG200_CA52,ANIEX300_,ANICP600_,ANIID300_,ANICP500_,ANSTO100_,ANIID300_,ANMOR800_,ANCVD400_P6,ANSPG200_CA42,ANIEX200_,ANICP600_,ANSTO400_,ANCVD400_P7,ANCLA200_,ANRTP100_,ANIEX800_,ANICP800_,ANSTO400_,ANSPS200_CA42,ANIEX500_,ANICP300_,ANSTO300_,ANMOR800_,ANANS200_,ANIEX100_,ANUSC100_,ANANO100_,ANCVD100_P4,ANCLA100_,ANIEX900_,ANICPA00_,ANSTO200_,...,SPS-04C_BC01,CAR-24C_C01,WA-05C_C01,WR-07C_C01,NaN,VDN-07C_CHB,CAJ-03C_C01,XO-07C_C02,VDN-07C_CHB,CAR-11C_C01,DRG-03C_CC01,WR-07C_C01,KO-45C_,EB-16C_C01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANDMY200_,FI-DUMY4_,FI-DUMY4_,ANDMY200_,NaN,NaN,NaN,NaN,ANAPK100_,_
EE1NB2K5H,EE01NB2K50P08,ANCLI100_,ANSPG100_CA42,ANIEX300_,ANWMA200_,ANSTO100_,ANIID300_,ANCVD600_P2,ANSPG200_CA42,ANIEX300_,ANICP600_,ANIID300_,ANICP500_,ANSTO100_,ANIID300_,ANMOR800_,ANCVD400_P7,ANSPG200_CA52,ANIEX200_,ANICP600_,ANSTO400_,ANCVD400_P2,ANCLA200_,ANRTP10

In [0]:
ARRAY_OPERATOR_DF[ARRAY_OPERATOR_DF['L6B_SHEET_ID']=='EE02N20600P01']['opid_key'].to_list()

Out[11]: ['08900_PEP10_ARY-FI',
 '03588_PEP10_AR-EB',
 '03567_PEP10_BP4-STO',
 '03560_PEP10_BP4-RIE',
 '03539_PEP10_BP4-IEX',
 '03532_PEP10_BP4-CVD',
 '03511_PEP9_PL4-ANI',
 '03490_PEP9_PL4-IEX',
 '03483_PEP9_PV3-CVD',
 '03392_PEP7_M5-STO',
 '03385_PEP7_M5-WMA',
 '03364_PEP7_M5-IEX',
 '03357_PEP7_M5-SPT',
 '03343_PEP6_BP3-STO',
 '03343_PEP6_BP3-STO',
 '03336_PEP6_BP3-RIE',
 '03315_PEP6_BP3-IEX',
 '03308_PEP6_BP3-CVD',
 '03287_PEP5_PL3-ANI',
 '03259_PEP5_PV2-CVD',
 '03217_PEP4_M4-STO',
 '03210_PEP4_M4-WMA',
 '03189_PEP4_M4-IEX',
 '03182_PEP4_M4-SPT',
 '03168_PEP3_BP2-STO',
 '03161_PEP3_BP2-RIE',
 '03140_PEP3_BP2-IEX',
 '03133_PEP3_BP2-CVD',
 '03112_PEP2_PL2-ANI',
 '03091_PEP2_PL2-IEX',
 '03000_PEP1_UKE-STO',
 '10000_-_ARY-APK',
 '09009_ANDMY200_ECS-BANK',
 '08888_-_ARY-FI',
 '00970_PEP9_BP2-IEX2',
 '00960_PEP9_BP2-CVD',
 '00950_PEP9_M33-TEG',
 '00945_PEP9_M33-TAR',
 '00943_PEP9_M33-ANI',
 '00942_PEP9_M33-TAR',
 '00941_PEP9_M33-ASO',
 '00940_PEP9_M33-STO',
 '00930_PEP9_M33-WMA',
 '00920_PEP9_M33-IEX',
 '00910_PEP9_M33-SPT',
 '00900_PEP9_M33-CLA',
 '00830_PEP8_BP-STO',
 '00820_PEP8_BP-ICP',
 '00810_PEP8_BP-IEX',
 '00805_PEP8_BP-STO0',
 '00800_PEP8_BP-CVD',
 '00710_PEP7_PL-ANI',
 '00705_PEP7_PL-USC',
 '00700_PEP7_PL-IEX',
 '00650_PEP6_SD-ANI',
 '00640_PEP6_SD-ASO',
 '00630_PEP6_SD-STO',
 '00620_PEP6_SD-ICP',
 '00610_PEP6_SD-IEX',
 '00600_PEP6_SD-SPT',
 '00570_PEP5_ILD-STO',
 '00560_PEP5_ILD-ICP',
 '00550_PEP5_ILD-IEX',
 '00540_PEP5_ILD-RTP',
 '00510_PEP5_ILD-CLA',
 '00500_PEP5_ILD-CVD',
 '00450_PEP4_M15-STO',
 '00420_PEP4_M15-ICP',
 '00400_PEP4_M15-SPT',
 '00390_PEP4_GI2-CVD',
 '00370_PEP3_GE-AIO2',
 '00360_PEP3_GE-BID2',
 '00350_PEP3_GE-STO',
 '00340_PEP3_GE-ICP2',
 '00330_PEP3_GE-BID',
 '00320_PEP3_GE-ICP',
 '00310_PEP3_GE-IEX',
 '00300_PEP3_GE-SPT',
 '00260_PEP2_GI-CVD',
 '00250_PEP2_NCD-IID',
 '00140_PEP1_SM-STO',
 '00130_PEP1_SM-ICP',
 '00120_PEP1_SM-IEX',
 '00110_PEP1_SM-SPT',
 '00100_PEP1_SM-CLA']

In [0]:
# SQL SEARCH AND CREATE DATAFRAME 查詢與產生資料表:
L6K_sheet_str = str(tuple(df_sheet_search['L6K_SHEET_ID'].to_list()))
L6B_sheet_str = str(tuple(df_sheet_search['L6B_SHEET_ID'].to_list()))

sql_string = f'''
  
  select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT, B.P_GOOD, B.LINE_DEF_CNT, B.POINT_DEF_CNT, A.REASON1_JUDGE, A.BIN_CNT, A.RANK1 
  from (
    select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT, A.REASON1_JUDGE, count(A.PANEL_ID) as BIN_CNT, 
    RANK() OVER(PARTITION BY A.SHEET_ID ORDER BY A.LOT_ETIME desc, count(A.PANEL_ID) desc, A.REASON1_JUDGE) RANK1
    from uled_analysis.mf_l6b_s_ary_at_mamcb1_hungdechen_test01 A 
    where A.SHEET_ID in {L6B_sheet_str} and A.REASON1_JUDGE not in ('')  
    group by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT, A.REASON1_JUDGE
  ) A LEFT JOIN 
    (
    select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT, sum(A.P_GOOD) as P_GOOD , sum(A.LINE_DEF_CNT) as LINE_DEF_CNT, sum(A.POINT_DEF_CNT) as POINT_DEF_CNT,
    RANK() OVER(PARTITION BY A.SHEET_ID ORDER BY A.LOT_ETIME desc) RANK1
    from uled_analysis.mf_l6b_s_ary_at_mamcb1_hungdechen_test01 A 
    where A.SHEET_ID in {L6B_sheet_str} and A.REASON1_JUDGE not in ('') 
    group by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT
  ) B 
  ON A.SHEET_ID=B.SHEET_ID and A.OP_NO=B.OP_NO and A.INS_EQPID=B.INS_EQPID and A.LOT_ETIME=B.LOT_ETIME and A.PANEL_CNT=B.PANEL_CNT
  where A.RANK1<6 and B.RANK1=1 
  order by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.LOT_ETIME, A.PANEL_CNT, A.RANK1 
  
'''

#print(sql_string)

df_data = spark.sql(sql_string) # pyspark dataframe

#df_data.write.csv("dbfs:/FileStore/result.csv")

df_L6B_AT = df_data.toPandas() # pandas dataframe
df_L6B_AT

Out[23]:

,SHEET_ID,OP_NO,INS_EQPID,LOT_ETIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,REASON1_JUDGE,BIN_CNT,RANK1
0,EE01NB2K50P07,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,2,0,29,46,1
1,EE01NB2K50P07,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,2,0,30,2,2
2,EE01NB2K50P08,4157,EB-16C,2022-04-20 20:04:27,48.0,47.0,2,0,29,47,1
3,EE01NB2K50P08,4157,EB-16C,2022-04-20 20:04:27,48.0,47.0,2,0,30,1,2
4,EE01NB2K50P09,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,50,0,29,46,1
...,...,...,...,...,...,...,...,...,...,...,...
2214,EE02N20600P04,3343,EB-16C,2022-04-27 14:03:22,48.0,43.0,24,1236,1,39,1
2215,EE02N20600P04,3343,EB-16C,2022-04-27 14:03:22,48.0,43.0,24,1236,27,4,2
2216,EE02N20600P04,3343,EB-16C,2022-04-27 14:03:22,48.0,43.0,24,1236,21,2,3
2217,EE02N20600P04,3343,EB-16C,2022-04-27 14:03:22,48.0,43.0,24,1236,25,1,4


In [0]:
ARRAY_L6BAT_DF = pd.merge(df_L6B_AT, df_SHEET_DF, left_on="SHEET_ID", right_on="L6K_SHEET_ID", how="inner").append(pd.merge(df_L6B_AT, df_SHEET_DF, left_on="SHEET_ID", right_on="L6B_SHEET_ID", how="inner"))[['L6K_SHEET_ID', 'L6B_SHEET_ID', 'SHEET_ID', 'OP_NO', 'INS_EQPID', 'LOT_ETIME', 'PANEL_CNT', 'P_GOOD', 'LINE_DEF_CNT', 'POINT_DEF_CNT', 'REASON1_JUDGE', 'BIN_CNT', 'RANK1']].sort_values(by=["L6K_SHEET_ID", "LOT_ETIME"],ascending=[True,False]).reset_index(drop=True)

test_df = df_sheet_search.merge(ARRAY_L6BAT_DF.pivot_table(values='BIN_CNT', index=['L6K_SHEET_ID', 'OP_NO', 'INS_EQPID', 'LOT_ETIME', 'PANEL_CNT', 'P_GOOD', 'LINE_DEF_CNT', 'POINT_DEF_CNT'], columns='REASON1_JUDGE', aggfunc='sum').reset_index(drop=False), how="left", on="L6K_SHEET_ID")
test_df

Out[51]:

,DVT,L6K_SHEET_ID,L6B_SHEET_ID,OP_NO,INS_EQPID,LOT_ETIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,1,10,21,22,23,25,26,27,28,29,30,5,7,9
0,4-5_mon,EE1NB2K5G,EE01NB2K50P07,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,2.0,NaN,NaN,NaN
1,4-5_mon,EE1NB2K5H,EE01NB2K50P08,4157,EB-16C,2022-04-20 20:04:27,48.0,47.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,1.0,NaN,NaN,NaN
2,4-5_mon,EE1NB2K5J,EE01NB2K50P09,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,50.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,2.0,NaN,NaN,NaN
3,4-5_mon,EE1NB2K5K,EE01NB2K50P10,4157,EB-16C,2022-04-20 20:04:27,48.0,46.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,2.0,NaN,NaN,NaN
4,4-5_mon,EE1NB2K5L,EE01NB2K50P11,4157,EB-16C,2022-04-20 20:04:27,48.0,48.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,4-7-2_MON2_T10V2+MEV1,EE2N20592,EE02N20590P26,3588,EB-16C,2022-05-14 20:01:38,48.0,36.0,0.0,17.0,32.0,NaN,7.0,NaN,3.0,1.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
614,4-7-2_MON2_T10V2+MEV1,EE2N2060A,EE02N20600P01,3343,EB-16C,2022-04-27 14:03:22,48.0,31.0,16.0,524.0,31.0,2.0,6.0,NaN,1.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
615,4-7-2_MON2_T10V2+MEV1,EE2N2060B,EE02N20600P02,3343,EB-16C,2022-04-27 14:03:22,48.0,16.0,235.0,2752.0,16.0,NaN,17.0,6.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616,4-7-2_MON2_T10V2+MEV1,EE2N2060C,EE02N20600P03,3343,EB-16C,2022-04-27 14:03:22,48.0,45.0,1.0,121.0,42.0,NaN,NaN,NaN,NaN,1.0,NaN,3.0,1.0,NaN,NaN,1.0,NaN,NaN


In [0]:
print(test_df['OP_NO'].unique())

test_df[test_df['OP_NO'].isna()]



['4157' '3588' '3589' '9001' nan '3427' '3343' '8900']
Out[48]:

,DVT,L6K_SHEET_ID,L6B_SHEET_ID_x,L6B_SHEET_ID_y,SHEET_ID,OP_NO,INS_EQPID,LOT_ETIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,1,10,21,22,23,25,26,27,28,29,30,5,7,9
239,4-7-2_MON2_T10V2+MEV1,EE2N13S1M,EE02N13S10P12,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,4-7-2_MON2_T10V2+MEV1,EE2N13S1N,EE02N13S10P13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,4-7-2_MON2_T10V2+MEV1,EE2N13S1P,EE02N13S10P14,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,4-7-2_MON2_T10V2+MEV1,EE2N13S1Q,EE02N13S10P15,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,4-7-2_MON2_T10V2+MEV1,EE2N13S1R,EE02N13S10P16,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,4-7-2_MON2_T10V2+MEV1,EE2N13S1S,EE02N13S10P17,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,4-7-2_MON2_T10V2+MEV1,EE2N13S1T,EE02N13S10P18,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,4-7-2_MON2_T10V2+MEV1,EE2N13S1U,EE02N13S10P19,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,4-7-2_MON2_T10V2+MEV1,EE2N13S1V,EE02N13S10P20,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,4-7-2_MON2_T10V2+MEV1,EE2N13S1W,EE02N13S10P21,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
%sql
    select A.* 
    from uled_analysis.mf_l6b_s_ary_at_mamcb1_hungdechen_test01 A 
    where A.SHEET_ID in ('EE02N13S10P12','EE02N20590P19') 


KIBAN_SEQ,PANEL_NO,LOT_ID,SHEET_ID,PANEL_ID,QUALITY_JUDGE,REASON1_JUDGE,REASON2_JUDGE,OP_NO,LOT_ETIME,SHEET_STIME,SHEET_ETIME,UPD_DATE,INS_EQPID,PANEL_CNT,K_JUDGE,M_COMMENT,SLOT_NO,D_INVAL_FLG,FURYO_CAUSE,DEV_RECIPE,P_GOOD,P_BAD,P_REWORK,P_OTHER,P_SKIP,G_OPEN_CNT,CS_OPEN_CNT,GG_SHORT_CNT,GCS_SHORT_CNT,S_OPEN_CNT,SS_SHORT_CNT,SG_SHORT_CNT,SCS_SHORT_CNT,G_OTHER_CNT,S_OTHER_CNT,T_1_CNT,T_2_CNT,T_3_CNT,CL_CNT,MU_CNT,REPAIR_CNT,SI_CNT,LINE_DEF_CNT,POINT_DEF_CNT,INS_DATE,INS_WEEK,INS_MONTH


In [0]:
ARRAY_L6BAT_DF = pd.merge(df_L6B_AT, df_SHEET_DF, left_on="SHEET_ID", right_on="L6K_SHEET_ID", how="inner").append(pd.merge(df_L6B_AT, df_SHEET_DF, left_on="SHEET_ID", right_on="L6B_SHEET_ID", how="inner"))[['L6K_SHEET_ID', 'L6B_SHEET_ID', 'SHEET_ID', 'OP_NO', 'INS_EQPID', 'LOT_ETIME', 'PANEL_CNT', 'P_GOOD', 'LINE_DEF_CNT', 'POINT_DEF_CNT', 'REASON1_JUDGE', 'BIN_CNT', 'RANK1']].sort_values(by=["L6K_SHEET_ID", "LOT_ETIME"],ascending=[True,False]).reset_index(drop=True)

ARRAY_L6BAT_DF['DFT_NAME'] = 'DFT_TOP' + ARRAY_L6BAT_DF['RANK1'].astype(str) + '_NAME'

ARRAY_L6BAT_DF.pivot_table(values='REASON1_JUDGE', index=['L6K_SHEET_ID', 'L6B_SHEET_ID', 'SHEET_ID', 'OP_NO', 'INS_EQPID', 'LOT_ETIME', 'PANEL_CNT', 'P_GOOD', 'LINE_DEF_CNT', 'POINT_DEF_CNT'], columns='DFT_NAME', aggfunc='sum').reset_index(drop=False)


Out[128]:

DFT_NAME,L6K_SHEET_ID,L6B_SHEET_ID,SHEET_ID,OP_NO,INS_EQPID,LOT_ETIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,DFT_TOP1_NAME,DFT_TOP2_NAME,DFT_TOP3_NAME,DFT_TOP4_NAME,DFT_TOP5_NAME
0,EE1NB2K51,EE01NB2K50P25,EE01NB2K50P25,3427,EB-16C,2022-03-20 10:54:08,48.0,42.0,299,1384,1,27,21,23,5
1,EE1NB2K51,EE01NB2K50P25,EE01NB2K50P25,3588,EB-16C,2022-03-28 10:37:47,48.0,40.0,305,1477,1,27,22,21,23
2,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,3427,EB-16C,2022-03-20 10:54:08,48.0,43.0,604,1963,1,21,22,23,27
3,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,3588,EB-16C,2022-03-28 10:37:47,48.0,42.0,4,494,1,21,22,23,26
4,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,4157,EB-16C,2022-04-20 10:57:05,48.0,47.0,434,0,29,30,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,EE2N2059Z,EE02N20590P24,EE02N20590P24,3588,EB-16C,2022-05-14 20:01:38,48.0,40.0,4,519,1,23,21,27,28
1414,EE2N2060A,EE02N20600P01,EE02N20600P01,3343,EB-16C,2022-04-27 14:03:22,48.0,31.0,16,524,1,25,21,10,23
1415,EE2N2060B,EE02N20600P02,EE02N20600P02,3343,EB-16C,2022-04-27 14:03:22,48.0,16.0,235,2752,21,1,22,23,25
1416,EE2N2060C,EE02N20600P03,EE02N20600P03,3343,EB-16C,2022-04-27 14:03:22,48.0,45.0,1,121,1,27,25,28,5


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2823504450892842> in <module> 
 ----> 1 ARRAY_L6BAT_DF . pivot_table ( values = 'BIN_CNT' , index = [ 'L6K_SHEET_ID' , 'L6B_SHEET_ID' , 'SHEET_ID' , 'OP_NO' , 'INS_EQPID' , 'LOT_ETIME' , 'PANEL_CNT' , 'P_GOOD' , 'LINE_DEF_CNT' , 'POINT_DEF_CNT' ] , columns = 'REASON1_JUDGE' , aggfunc = 'sum' ) . reset_index ( drop = False ) 

 NameError : name 'ARRAY_L6BAT_DF' is not defined

In [0]:
ARRAY_L6BAT_DF['DFT_VALUE'] = 'DFT_TOP' + ARRAY_L6BAT_DF['RANK1'].astype(str) + '_VALUE'

ARRAY_L6BAT_DF.pivot_table(values='BIN_CNT', index=['L6K_SHEET_ID', 'L6B_SHEET_ID', 'SHEET_ID', 'OP_NO', 'INS_EQPID', 'LOT_ETIME', 'PANEL_CNT', 'P_GOOD', 'LINE_DEF_CNT', 'POINT_DEF_CNT'], columns='DFT_VALUE', aggfunc='sum').reset_index(drop=False)

Out[127]:

DFT_VALUE,L6K_SHEET_ID,L6B_SHEET_ID,SHEET_ID,OP_NO,INS_EQPID,LOT_ETIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,DFT_TOP1_VALUE,DFT_TOP2_VALUE,DFT_TOP3_VALUE,DFT_TOP4_VALUE,DFT_TOP5_VALUE
0,EE1NB2K51,EE01NB2K50P25,EE01NB2K50P25,3427,EB-16C,2022-03-20 10:54:08,48.0,42.0,299,1384,37.0,5.0,3.0,2.0,1.0
1,EE1NB2K51,EE01NB2K50P25,EE01NB2K50P25,3588,EB-16C,2022-03-28 10:37:47,48.0,40.0,305,1477,36.0,4.0,3.0,2.0,1.0
2,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,3427,EB-16C,2022-03-20 10:54:08,48.0,43.0,604,1963,42.0,3.0,1.0,1.0,1.0
3,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,3588,EB-16C,2022-03-28 10:37:47,48.0,42.0,4,494,41.0,3.0,1.0,1.0,1.0
4,EE1NB2K52,EE01NB2K50P26,EE01NB2K50P26,4157,EB-16C,2022-04-20 10:57:05,48.0,47.0,434,0,47.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,EE2N2059Z,EE02N20590P24,EE02N20590P24,3588,EB-16C,2022-05-14 20:01:38,48.0,40.0,4,519,38.0,3.0,2.0,2.0,2.0
1414,EE2N2060A,EE02N20600P01,EE02N20600P01,3343,EB-16C,2022-04-27 14:03:22,48.0,31.0,16,524,31.0,8.0,6.0,2.0,1.0
1415,EE2N2060B,EE02N20600P02,EE02N20600P02,3343,EB-16C,2022-04-27 14:03:22,48.0,16.0,235,2752,17.0,16.0,6.0,5.0,4.0
1416,EE2N2060C,EE02N20600P03,EE02N20600P03,3343,EB-16C,2022-04-27 14:03:22,48.0,45.0,1,121,42.0,3.0,1.0,1.0,1.0


In [0]:
%sql

    select A.*
    from uledgp_eng.l6k_array_archip A 
    where A.SHEET_ID in ('EE2N2059S','EE2N13S1S') and A.GNG_FRG not in ('') 


T_STAMP,TOOL_TYPE,TOOL_ID,TEST_CASE,LOT_ID,CHIP_ID,DDATE,DDATE_TTIME,OPERATOR_ID,LOT_YLD,GNG_FRG,SHORT_DFT_NO,OPEN_DFT_NO,POINT_DFT_NO,AVG_P_MODE,AVG_N_MODE,MAX_P_MODE,MAX_N_MODE,MIN_P_MODE,MIN_N_MODE,MAX_P_AVG,MAX_N_AVG,MIN_P_AVG,MIN_N_AVG,PATTERN_ID,REICPE_ID,PF_ID,STATUS_OF_PD,G_BP_NO,BP_NO,DP_NO,SIG_OPEN_NO,GAT_OPEN_NO,CS_OPEN_NO,SS_SHORT_NO,SG_SORT_NO,SCS_SHORT_NO,GG_SHORT_NO,GCS_SHORT_NO,SIG_SHORT_NO,GAT_SHORT_NO,PT_DFT_NO,P1_NO,P2_NO,P3_NO,SIG_OPEN_CHIP_LOSS,GAT_OPEN_CHIP_LOSS,CS_OPEN_CHIP_LOSS,SS_SHORT_CHIP_LOSS,SG_SHORT_CHIP_LOSS,SCS_SHORT_CHIP_LOSS,GG_SHORT_CHIP_LOSS,GCS_SHORT_CHIP_LOSS,P1_CHIP_LOSS,P2_CHIP_LOSS,P3_CHIP_LOSS,OTHERS_CHIP_LOSS,SIG_SHORT_CHIP_LOSS,GAT_SHORT_CHIP_LOSS,NEW_FLAG,CHIP_SEQ_NO,P4_CHIP_LOSS,P4_NO,SLOT_NO,SHEET_ID,MODEL_NO,ABBR_NO,CST_ID,MATERIAL,DOWNLOAD_LINK_KEY,TESTER_OPID,LOT_START_TIME,LOT_END_TIME,SHEET_START_TIME,SHEET_END_TIME,CHIP_END_TIME,PROBE_CARD_ID,ESCAPE_LINE_SOURCE_NO,ESCAPE_LINE_GATE_NO,ESCAPE_AREA_NO,MIN_VOLTAGE,MAX_VOLTAGE,AVG_VOLTAGE,AVG_R_VOLTAGE,AVG_G_VOLTAGE,AVG_B_VOLTAGE,VOLTAGE_UPPER_LIMIT,VOLTAGE_LOWER_LIMIT,CHARGE_MAP_PATH,CHARGE_MAP_FILENAME,D1_NO,D2_NO,D3_NO,D4_NO,PIN_FAIL_NO,VID_SHRT_NO,VID_OPEN_NO,DRV_SHRT_NO,DRV_OPEN_NO,POW_FAIL_NO,SR_FAIL_NO,D1_CHIP_LOSS,D2_CHIP_LOSS,D3_CHIP_LOSS,D4_CHIP_LOSS,PIN_FAIL_CHIP_LOSS,VID_SHRT_CHIP_LOSS,VID_OPEN_CHIP_LOSS,DRV_SHRT_CHIP_LOSS,DRV_OPEN_CHIP_LOSS,POW_FAIL_CHIP_LOSS,SR_FAIL_CHIP_LOSS,CHIP_BIN,SHEET_YIELD,ITP_AVG_VOLTAGE,ITP_SHEET_YIELD,ITP_LOT_YIELD,ITP_OPEN_NO,ITP_SHORT_NO,ITP_OPEN_CHIP_LOSS,ITP_SHORT_CHIP_LOSS,ITP_CHARGE_MAP_PATH,ITP_CHARGE_MAP_FILENAME,ITP_SPEC1,ITP_SPEC2,ITP_SPEC3,ITP_JUDGE_TYPE,ITP_CHARGE_MAP2_PATH,ITP_CHARGE_MAP2_FILENAME,ITP_BRIGHT_SENSOR_AVG_CAP,ITP_BLACK_SENSOR_AVG_CAP,ITP_BRIGHT_SENSOR_AVG_CAP2,ITP_BLACK_SENSOR_AVG_CAP2
2022-03-14T16:39:45.000+0000,TAR,ANTAR700,S,EE2N13S100,EE2N13S1S0AA,20220314,2022-03-14T13:39:35.000+0000,M33-TAR,0.0,1,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,T127V270,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,Y,43,0.0,0.0,17,EE2N13S1S,T127S1-N3,EE,AA0983,null,1285,M33-TAR,2022-03-14T08:27:16.000+0000,2022-03-14T16:38:12.000+0000,2022-03-14T13:31:59.000+0000,2022-03-14T13:49:56.000+0000,2022-03-14T13:40:43.000+0000,1,1.0,0.0,0.0,39.1,748.4,379.4,null,null,null,500.0,30.0,\EE2N13S100\EE2N13S1S,EE2N13S1S0AA.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O,39.58,null,100.0,null,0.0,0.0,0.0,0.0,\EE2N13S100\EE2N13S1S,EE2N13S1S0AA_Set1.jpg,null,null,null,null,\EE2N13S100\EE2N13S1S,EE2N13S1S0AA_Set2.jpg,null,null,null,null
2022-03-14T16:39:45.000+0000,TAR,ANTAR700,S,EE2N13S100,EE2N13S1S0AB,20220314,2022-03-14T13:38:20.000+0000,M33-TAR,0.0,1,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,T127V270,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,Y,37,0.0,0.0,17,EE2N13S1S,T127S1-N3,EE,AA0983,null,1285,M33-TAR,2022-03-14T08:27:16.000+0000,2022-03-14T16:38:12.000+0000,2022-03-14T13:31:59.000+0000,2022-03-14T13:49:56.000+0000,2022-03-14T13:39:29.000+0000,1,1.0,0.0,0.0,58.8,704.7,353.6,null,null,null,500.0,30.0,\EE2N13S100\EE2N13S1S,EE2N13S1S0AB.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,O,39.58,null,100.0,null,0.0,0.0,0.0,0.0,\EE2N13S100\EE2N13S1S,EE2N13S1S0AB_Set1.jpg,null,null,null,null,\EE2N13S100\EE2N13S1S,EE2N13S1S0AB_Set2.jpg,null,null,null,null
2022-03-14T16:39:45.000+0000,TAR,ANTAR700,S,EE2N13S100,EE2N13S1S0AC,20220314,2022-03-14T13:37:14.000+0000,M33-TAR,0.0,27,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,T127V270,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,Y,31,0.0,0.0,17,EE2N13S1S,T127S1-N3,EE,AA0983,null,1285,M33-TAR,2022-03-14T08:27:16.000+0000,2022-03-

In [0]:
%sql

    select A.SHEET_ID, A.TESTER_OPID as OP_NO, A.TOOL_ID as INS_EQPID, A.LOT_END_TIME as LOT_ETIME, 48 as PANEL_CNT, A.GNG_FRG, 
    case when A.CHIP_BIN='X' then 0 else 1 end as P_GOOD , A.SHORT_DFT_NO+A.OPEN_DFT_NO as LINE_DEF_CNT, A.POINT_DFT_NO as POINT_DEF_CNT,
    RANK() OVER(PARTITION BY A.SHEET_ID ORDER BY A.LOT_END_TIME desc) RANK1
    from uledgp_eng.l6k_array_archip A 
    where A.SHEET_ID in ('EE2N2059S','EE2N13S1S') and A.GNG_FRG not in ('') 

In [0]:
%sql
select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT, B.P_GOOD, B.LINE_DEF_CNT, B.POINT_DEF_CNT, A.REASON1_JUDGE, A.BIN_CNT, A.RANK1 
from (
select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT, A.REASON1_JUDGE, count(A.PANEL_ID) as BIN_CNT, 
RANK() OVER(PARTITION BY A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT ORDER BY count(A.PANEL_ID) desc, A.REASON1_JUDGE) RANK1
from uled_analysis.mf_l6b_s_ary_at_mamcb1_hungdechen_test01 A 
where A.SHEET_ID in ('EE01NB2K50P25', 'EE02N20600P01') 
group by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT, A.REASON1_JUDGE
) A LEFT JOIN 
(
select A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT, sum(A.P_GOOD) as P_GOOD , sum(A.LINE_DEF_CNT) as LINE_DEF_CNT, sum(A.POINT_DEF_CNT) as POINT_DEF_CNT  
from uled_analysis.mf_l6b_s_ary_at_mamcb1_hungdechen_test01 A 
where A.SHEET_ID in ('EE01NB2K50P25', 'EE02N20600P01') 
group by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT
) B 
ON A.SHEET_ID=B.SHEET_ID and A.OP_NO=B.OP_NO and A.INS_EQPID=B.INS_EQPID and A.SHEET_STIME=B.SHEET_STIME and A.PANEL_CNT=B.PANEL_CNT
where RANK1<6 
order by A.SHEET_ID, A.OP_NO, A.INS_EQPID, A.SHEET_STIME, A.PANEL_CNT, A.RANK1 

SHEET_ID,OP_NO,INS_EQPID,SHEET_STIME,PANEL_CNT,P_GOOD,LINE_DEF_CNT,POINT_DEF_CNT,REASON1_JUDGE,BIN_CNT,RANK1
EE01NB2K50P25,3427,EB-16C,2022-03-20T10:04:40.000+0000,48.0,42.0,299,1384,1,37,1
EE01NB2K50P25,3427,EB-16C,2022-03-20T10:04:40.000+0000,48.0,42.0,299,1384,27,5,2
EE01NB2K50P25,3427,EB-16C,2022-03-20T10:04:40.000+0000,48.0,42.0,299,1384,21,3,3
EE01NB2K50P25,3427,EB-16C,2022-03-20T10:04:40.000+0000,48.0,42.0,299,1384,23,2,4
EE01NB2K50P25,3427,EB-16C,2022-03-20T10:04:40.000+0000,48.0,42.0,299,1384,5,1,5
EE01NB2K50P25,3588,EB-16C,2022-03-28T09:48:10.000+0000,48.0,40.0,305,1477,1,36,1
EE01NB2K50P25,3588,EB-16C,2022-03-28T09:48:10.000+0000,48.0,40.0,305,1477,27,4,2
EE01NB2K50P25,3588,EB-16C,2022-03-28T09:48:10.000+0000,48.0,40.0,305,1477,22,3,3
EE01NB2K50P25,3588,EB-16C,2022-03-28T09:48:10.000+0000,48.0,40.0,305,1477,21,2,4
EE01NB2K50P25,3588,EB-16C,2022-03-28T09:48:10.000+0000,48.0,40.0,305,1477,23,1,5


In [0]:
# SQL SEARCH AND CREATE DATAFRAME 查詢與產生資料表:

sql_string = f'''
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, A.PEP_LEVEL, A.OP_ID, A.OP_SEQ, A.RECIPE_ID, A.ACTUAL_PPID, A.EQP_ID, A.CHAMBER_ID_LIST, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6b_array_h_sheet_oper_ods A 
  where A.SHEET_ID_CHIP_ID in 
  ( 
    select SOURCE_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by SOURCE_SHEET_ID 
  )  
  or A.SHEET_ID_CHIP_ID in 
  ( 
    select TARGET_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by TARGET_SHEET_ID 
  ) 
  
  union 
  
  select A.LOT_ID, A.SHEET_ID_CHIP_ID as SHEET_ID, A.MFG_DAY, A.MODEL_NO, A.PRODUCT_CODE, A.MAIN_ROUTE_ID, A.ROUTE_ID, A.PEP_LEVEL, A.OP_ID, A.OP_SEQ, A.RECIPE_ID, A.ACTUAL_PPID, A.EQP_ID, A.CHAMBER_ID_LIST, A.CASSETTE_ID, A.REWORK_FLAG, A.ALARM_FLAG, A.LOGON_TIME, A.LOGOFF_TIME, A.HOLD_TIME, A.QUEUE_TIME, A.OP_PROCESS_TIME, A.OP_CYCLE_TIME, A.EQP_RUN_ID_START_TIME, A.EQP_RUN_ID_END_TIME 
  from uledgp_prod.l6k_array_h_sheet_oper_ods A 
  where A.SHEET_ID_CHIP_ID in 
  ( 
    select SOURCE_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by SOURCE_SHEET_ID 
  )  
  or A.SHEET_ID_CHIP_ID in 
  ( 
    select TARGET_SHEET_ID 
    from uledgp_prod.l6k_array_h_dax_sheetchipmapping_ods 
    where TARGET_SHEET_ID in {sheet_str} or SOURCE_SHEET_ID in {sheet_str}
    group by TARGET_SHEET_ID 
  ) 
  
'''

#print(sql_string)

df_data = spark.sql(sql_string) # pyspark dataframe

#df_data.write.csv("dbfs:/FileStore/result.csv")

df_data_pandas = df_data.toPandas() # pandas dataframe
df_data_pandas

Out[5]:

,LOT_ID,SHEET_ID,MFG_DAY,MODEL_NO,PRODUCT_CODE,MAIN_ROUTE_ID,ROUTE_ID,PEP_LEVEL,OP_ID,OP_SEQ,RECIPE_ID,ACTUAL_PPID,EQP_ID,CHAMBER_ID_LIST,CASSETTE_ID,REWORK_FLAG,ALARM_FLAG,LOGON_TIME,LOGOFF_TIME,HOLD_TIME,QUEUE_TIME,OP_PROCESS_TIME,OP_CYCLE_TIME,EQP_RUN_ID_START_TIME,EQP_RUN_ID_END_TIME
0,EE12G306Y0,EE01NB2K50P07,2022-03-22,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP9,PV3-CVD,3483,P2EAPV30PV200000,P2EAPV30PV200000,VDN-07C,CHC,SA075405,N,None,2022-03-23 00:46:13,2022-03-23 01:58:38,0,16224,4345,20569,2022-03-23 01:06:46,NaT
1,EE22G407Z0,EE01NB2K50P08,2022-04-20,T127SUN01_T2_BP,T127SUN01,EE2_04,EE2_04,PEP15,BP6-ASO,4129,EE22440000000001,EE22440000000001,KO-45C,None,SA632177,N,None,2022-04-20 09:46:36,2022-04-20 12:14:36,0,15119,8880,23999,2022-04-20 11:12:10,NaT
2,EE12G306Y0,EE01NB2K50P08,2022-03-22,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP8,ITO1-STO,3456,0000060100000000,0000060100000000,WR-10C,C01,SA075405,N,None,2022-03-22 15:46:47,2022-03-22 16:24:25,0,3283,2258,5541,2022-03-22 15:55:08,NaT
3,EE12G306Y0,EE01NB2K50P07,2022-03-04,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP1,UKE-STO,3000,0000060100000000,0000060100000000,WR-10C,C01,SA075405,N,None,2022-03-04 08:57:16,2022-03-04 09:22:44,0,41226,1528,42754,2022-03-04 08:57:25,NaT
4,EE12G306Y0,EE01NB2K50P07,2022-03-07,T127SUN01_T2_FP,T127SUN01,EE1_03,EE1_03,PEP3,BP2-IEX,3140,1630016300000000,1630016300000000,CAR-05C,C01,SA075405,N,None,2022-03-07 17:35:26,2022-03-07 19:08:26,0,80666,5580,86246,2022-03-07 17:35:55,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,EE1NB2K500,EE1NB2K5G,2021-12-11,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP4,M15-ICP,420,T127SUN01-GE1.5,T127SUN01-GE1.5,ANICP600,2,AA0904,N,None,2021-12-11 18:10:51,2021-12-11 20:21:05,0,19164,7814,26978,2021-12-11 19:31:38,NaT
250,EE1NB2K500,EE1NB2K5G,2021-11-25,T127S1-N3,T127SUN01,T127S1EM,RT127UN2T04,PEP2,POLY+CL2,2131,RWK-CLA,RWK-CLA,ANALK100,None,AA0904,Y,None,2021-11-25 15:05:08,2021-11-25 16:05:51,0,479718,3643,483361,2021-11-25 15:23:25,NaT
251,EE1NB2K500,EE1NB2K5G,2021-11-16,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP1,SM-CLA,100,CN5TV1,CN5TV1,ANCLI100,None,AA0904,N,None,2021-11-16 15:07:30,2021-11-16 15:38:57,0,0,1887,0,2021-11-16 15:13:54,NaT
252,EE1NB2K500,EE1NB2K5H,2021-11-29,T127S1-N3,T127SUN01,T127S1EM,T127S1EM,PEP3,GE-SPT,300,LM3000-HR,LM3000-HR,ANSPG200,CA42,AA0904,N,None,2021-11-29 21:59:59,2021-11-29 23:27:18,0,145452,5239,150691,2021-11-29 22:45:03,NaT


In [0]:
df_data.write.csv("/dbfs/FileStore/result.csv")